## Query 3

> Para cada tipo de pago y segmento de cliente, devolver la suma y el promedio expresado como porcentaje, de clientes activos y de consentimiento de marketing. Se valora que el output de la consulta tenga nombres claros y en español.

Los pasos para ello sería:

1. Importar el dataset de clientes (data/raw/customers.csv).
2. Filtrar los clientes que no tienen valores válidos en las columnas "active", "marketing_consent" y "segment".
3. Mapear solamente los valores que nos interesan por cliente: "customer_id", "active", "marketing_consent" y "segment".
4. Importar el dataset de órdenes (data/raw/orders.csv).
5. Filtrar las órdenes que no tienen valores válidos en las columnas "payment_method" y "customer_id".
6. Mapear solamente los valores que nos interesan por orden: "order_id", "payment_method" y "customer_id".
7. Mapear los valores de "customer_id" y "payment_method" como claves y reducir por clave para filtrar potenciales filas duplicadas (un cliente puede tener varias órdenes con el mismo método de pago).
8. Hacer un join entre las órdenes y los clientes por "customer_id".
9. Mapear los segmentos de cliente y métodos de pago como claves y los valores de "active" y "marketing_consent" como valores.
10. Reducir por clave para obtener la cuenta total, cuenta de activos y cuenta de consentimientos de marketing.
11. Mapear los resultados para calcular el porcentaje de activos y consentimientos de marketing.
12. Aplicar un collect para obtener los resultados y mostrarlos. Esto es seguro ya que la cantidad de resultados sería proporcional a la cantidad de segmentos y métodos de pago, que es un número pequeño ya que lo estaremos acotando en el filtro inicial.

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MetodosPago").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
allowed_payment_methods = set(
    [
        "BANK TRANSFER",
        "CASH ON DELIVERY",
        "CREDIT CARD",
        "DEBIT CARD",
        "DIGITAL WALLET",
        "PAYPAL",
    ]
)
allowed_segments = set(["REGULAR", "PREMIUM", "BUDGET"])


def normalize_bool(val):
    if val is None:
        return None
    s = str(val).strip().lower()
    if s in ("true", "1", "yes", "y"):
        return True
    if s in ("false", "0", "no", "n"):
        return False
    return None


def normalize_from_set(val, allowed_set):
    if val is None:
        return None

    s = str(val).strip().upper()

    return s if s in allowed_set else None


def normalize_payment_method(pm):
    return normalize_from_set(pm, allowed_payment_methods)


def normalize_segment(seg):
    return normalize_from_set(seg, allowed_segments)

In [21]:
df_customers = spark.read.csv(
    "../../data/raw/customers.csv", header=True, inferSchema=True
)
rdd_customers = df_customers.rdd
rdd_customers_min = rdd_customers.map(
    lambda r: {
        "customer_id": getattr(r, "customer_id", None),
        "customer_segment": normalize_segment(getattr(r, "customer_segment", None)),
        "is_active": normalize_bool(getattr(r, "is_active", None)),
        "marketing_consent": normalize_bool(getattr(r, "marketing_consent", None)),
    }
).filter(
    lambda x: x["customer_id"] is not None
    and x["customer_segment"] is not None
    and x["is_active"] is not None
    and x["marketing_consent"] is not None
)

df_orders = spark.read.csv("../../data/raw/orders.csv", header=True, inferSchema=True)
rdd_orders = df_orders.rdd
orders_cleaned = rdd_orders.map(
    lambda r: {
        "customer_id": getattr(r, "customer_id", None),
        "payment_method": normalize_payment_method(getattr(r, "payment_method", None)),
    }
).filter(lambda x: x["customer_id"] is not None and x["payment_method"] is not None)

unique_orders = (
    orders_cleaned.map(
        lambda x: (
            (x["customer_id"], x["payment_method"]),
            {"customer_id": x["customer_id"], "payment_method": x["payment_method"]},
        )
    )
    .reduceByKey(lambda a, _: a)
    .map(lambda kv: kv[1])
)

In [28]:
rdd_customers_with_key = rdd_customers_min.map(
    lambda x: (
        x["customer_id"],
        {
            "customer_segment": x["customer_segment"],
            "is_active": x["is_active"],
            "marketing_consent": x["marketing_consent"],
        },
    )
)
rdd_orders_with_key = unique_orders.map(
    lambda x: (x["customer_id"], x["payment_method"])
)

mapped = rdd_orders_with_key.join(rdd_customers_with_key).map(
    lambda kv: (
        (kv[1][0], kv[1][1]["customer_segment"]),
        {
            "total": 1,
            "activos": 1 if kv[1][1]["is_active"] else 0,
            "consentidos": 1 if kv[1][1]["marketing_consent"] else 0,
        },
    )
)

reduced = mapped.reduceByKey(
    lambda a, b: {
        "total": a["total"] + b["total"],
        "activos": a["activos"] + b["activos"],
        "consentidos": a["consentidos"] + b["consentidos"],
    }
)

results = reduced.map(
    lambda kv: {
        "metodo_pago": kv[0][0],
        "segmento": kv[0][1],
        "total_clientes": kv[1]["total"],
        "activos": kv[1]["activos"],
        "consentidos": kv[1]["consentidos"],
        "pct_activos": (
            f"{round(kv[1]['activos'] / kv[1]['total'] * 100, 2)}%"
            if kv[1]["total"] > 0
            else "0.0%"
        ),
        "pct_consentidos": (
            f"{round(kv[1]['consentidos'] / kv[1]['total'] * 100, 2)}%"
            if kv[1]["total"] > 0
            else "0.0%"
        ),
    }
).collect()

In [37]:
import pandas as pd

df_res = pd.DataFrame(results)
if not df_res.empty:
    df_res = df_res[
        [
            "metodo_pago",
            "segmento",
            "total_clientes",
            "activos",
            "pct_activos",
            "consentidos",
            "pct_consentidos",
        ]
    ]
    display(df_res.sort_values(["metodo_pago", "segmento"]).reset_index(drop=True))
else:
    print("No hay resultados")

,metodo_pago,segmento,total_clientes,activos,pct_activos,consentidos,pct_consentidos
0,BANK TRANSFER,BUDGET,17867,16041,89.78%,12451,69.69%
1,BANK TRANSFER,PREMIUM,18292,16441,89.88%,12808,70.02%
2,BANK TRANSFER,REGULAR,54606,49126,89.96%,38297,70.13%
3,CASH ON DELIVERY,BUDGET,17863,16040,89.79%,12449,69.69%
4,CASH ON DELIVERY,PREMIUM,18288,16437,89.88%,12807,70.03%
5,CASH ON DELIVERY,REGULAR,54620,49140,89.97%,38307,70.13%
6,CREDIT CARD,BUDGET,17865,16039,89.78%,12448,69.68%
7,CREDIT CARD,PREMIUM,18286,16434,89.87%,12807,70.04%
8,CREDIT CARD,REGULAR,54614,49132,89.96%,38301,70.13%
9,DEBIT CARD,BUDGET,17860,16034,89.78%,12441,69.66%
